Олег Дмитриев

Эксплоратоный анализ данных представлен в блокноте с названием data_exploration.ipynb

# Выбор модели и метрик

Факторы определяющие выбор модели и метрики для оценки ее качества:

- Задача, поставленная в задании, требует от модели ответа на вопрос, а не выбора среди вариантов ответа. 

- Нужно научиться отвечать на вопрос основе предоставленных доĸументов. Вход: Вопрос + Доĸумент. Это задача не так сложна. Можно использовать генеративные модели, однако они требуют большого количества вычислительных ресурсов как на стадии обучения, так и на стадии инференса. Я остановился на модели которая находит ответ в контексте, а не генерирует его сама.

- Также нужно научиться отвечать на вопрос без предоставленного документа. Этот таск по сложнее. Предполагается, что это может быть вопрос по какому-то из документов датасета. В этом случае будем использовать библиотеку [langchain](https://www.langchain.com/) и [Annoy](https://python.langchain.com/docs/integrations/vectorstores/annoy) для индексации документов вытаскивания подходящих. Будем индексировать тренировочный датасет.

- Тк ответы не являются цитированием контекста (как в SQuAD), поэтому не подойдут метрики, основанные на пересечении слов в ответе и контексте. Придется находить косинусное расстояние (1 - cosine similarity) между векторами ответа данного моделью и векторами ответов из датасета. Модель дала правильный ответ, если косинусное расстояние между ответом модели и ЛЮБЫМ из  правильных ответов из датасета наименьшее. Тк модель ищет ответ на вопрос в контексте, то учесть несколько вариантов ответа на вопрос не получится. Поэтому я буду использовать только первый вариант ответа на вопрос из датасета. По определению это метрика accuracy.

- Также предлагается использовать метрику MRR (Mean Reciprocal Rank). Мы ранжируем косинусные расстояния между ответом модели и ответами из датасета. Reciprocal Rank - это обратное ранговое значение первого правильного ответа. Усредним эти значения по всем вопросам. Получим MRR.


Ниже можно ознакомиться с кодом, который реализует описанную выше логику. Готовый продукт представлен в streamlit приложении. Запустить его можно командой:

```shell
streamlit run streamlit_app.py
```

Для запуска потребуется создать виртуальное окружение и установить зависимости:

```shell
conda create --name biocad-ml-env python=3.11
conda activate biocad-ml-env
pip install -r requirements.txt
```

Альтернативой является использование [Dockerfile](Dockerfile)

* `docker build -t ml-biocad-image .`

* Запускаем контейнер, прикрепляем к нему папку с репозиторием и запускаем streamlit приложение
```shell
docker run -it --name ml-biocad-cont -v ПУТЬ_ДО_ПАПКИ_С_КОДОМ:/ml ml-biocad-image
cd /ml
streamlit run streamlit_app.py --server.port=8501 --server.address=0.0.0.0
```


In [2]:
import re
import torch
from transformers import pipeline
import json
import numpy as np 
from tqdm import tqdm   
from sklearn.metrics import top_k_accuracy_score
from sentence_transformers import SentenceTransformer, util
import torch
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Annoy

/home/langley/miniconda3/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2023-11-19 15:04:11.451862: I tensorflow/core/util/port.cc:111] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-11-19 15:04:11.473014: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-11-19 15:04:11.536766: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-11-19 15:04:11.536819: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable 

In [3]:
# load train dataset

ds = []
with open('data/train.jsonl', 'r') as f:
    for line in f:
        ds.append(json.loads(line))

# реформатирование датасета чтобы было удобнее работать
def format_dataset(example):
    context = example['passage']['text']
    context = re.sub(r'\(\d+\)', '', context) # remove (1), (2), etc.
    questions = [q['question'] for q in example['passage']['questions']]
    answers = []
    labels = []
    for q in example['passage']['questions']:
        answers.append([])
        labels.append([])
        for a in q['answers']:
            answers[-1].append(a['text'])
            labels[-1].append(a['label'])
    assert len(questions) == len(answers)
    list_of_data = []
    for i in range(len(questions)):
        if len(labels[i]) <= 6:
            list_of_data.append({
                'context': context,
                'question': questions[i], 
                'answers': answers[i], 
                'labels': labels[i]
            })
    return list_of_data

new_ds = []
for example in ds:
    new_ds.extend(format_dataset(example))

new_ds[2]

{'context': ' Но люди не могут существовать без природы, поэтому в парке стояли железобетонные скамейки — деревянные моментально ломали.  В парке бегали ребятишки, водилась шпана, которая развлекалась игрой в карты, пьянкой, драками, «иногда насмерть».  «Имали они тут и девок...»  Верховодил шпаной Артемка-мыло, с вспененной белой головой.  Людочка сколько ни пыталась усмирить лохмотья на буйной голове Артемки, ничего у неё не получалось.  Его «кудри, издали напоминавшие мыльную пену, изблизя оказались что липкие рожки из вокзальной столовой — сварили их, бросили комком в пустую тарелку, так они, слипшиеся, неподъёмно и лежали.  Да и не ради причёски приходил парень к Людочке.  Как только её руки становились занятыми ножницами и расчёской, Артемка начинал хватать её за разные места.  Людочка сначала увёртывалась от хватких рук Артемки, а когда не помогло, стукнула его машинкой по голове и пробила до крови, пришлось лить йод на голову «ухажористого человека».  Артемка заулюлюкал и со св

Используем модель [sentence-transformers/paraphrase-multilingual-mpnet-base-v2](https://huggingface.co/sentence-transformers/paraphrase-multilingual-mpnet-base-v2) для определения сродства между предложениями. Она предназначена для работы с русским языком и показала хороший результат в моих других проектах.

In [6]:
# выбираем девайс
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# загружаем модель для подсчета косинусного расстояния
embedder = SentenceTransformer('sentence-transformers/paraphrase-multilingual-mpnet-base-v2')

# загружаем пайплайн для предсказания ответа на вопрос
model_for_answering_name = "timpal0l/mdeberta-v3-base-squad2"
qa_pipeline = pipeline("question-answering", 
                       model=model_for_answering_name, 
                       tokenizer=model_for_answering_name,
                       device=device)


Пример работы пайплайна:

In [72]:
ind = 0

c = new_ds[ind]['context']
q = new_ds[ind]['question']

ans = qa_pipeline(question=q,  context=c)
print('Контекст: ', c)
print('Вопрос: ', q)
print('Ответ: ', ans['answer'])

Контекст:  (1) Но люди не могут существовать без природы, поэтому в парке стояли железобетонные скамейки — деревянные моментально ломали. (2) В парке бегали ребятишки, водилась шпана, которая развлекалась игрой в карты, пьянкой, драками, «иногда насмерть». (3) «Имали они тут и девок...» (4) Верховодил шпаной Артемка-мыло, с вспененной белой головой. (5) Людочка сколько ни пыталась усмирить лохмотья на буйной голове Артемки, ничего у неё не получалось. (6) Его «кудри, издали напоминавшие мыльную пену, изблизя оказались что липкие рожки из вокзальной столовой — сварили их, бросили комком в пустую тарелку, так они, слипшиеся, неподъёмно и лежали. (7) Да и не ради причёски приходил парень к Людочке. (8) Как только её руки становились занятыми ножницами и расчёской, Артемка начинал хватать её за разные места. (9) Людочка сначала увёртывалась от хватких рук Артемки, а когда не помогло, стукнула его машинкой по голове и пробила до крови, пришлось лить йод на голову «ухажористого человека». (1

Функция для вычисления косинусного расстояния между векторами

In [73]:

payload = {
    "inputs": {
        "source_sentence": ans['answer'],
        "sentences": new_ds[ind]['answers']
    }
}

def cosine_sim_local(payload : dict) -> np.ndarray:
    source = payload["inputs"]["source_sentence"]
    compare_to = payload["inputs"]["sentences"]
    source_inp = embedder.encode(source, convert_to_tensor=True)
    compare_inp = embedder.encode(compare_to, convert_to_tensor=True)
    return util.pytorch_cos_sim(source_inp, compare_inp)[0].cpu().numpy()


def cosine_distance(payload : dict)-> np.ndarray:
    return 1 - np.array(cosine_sim_local(payload))

cosine_distance(payload)

array([0.01532322, 0.3975693 , 0.5284183 ], dtype=float32)

Определим метрики accuracy и Reciprocal Rank

In [74]:
def is_correct(payload : dict, labels : list) -> bool:
    return labels[np.argmin(cosine_distance(payload))] == 1

def rr(payload : dict, labels : list) -> float:
    sorted_inds = cosine_distance(payload).argsort()
    for i, ind in enumerate(sorted_inds):
        if labels[ind] == 1:
            return 1 / (i + 1)
    return 0

Вычислим accuracy и MRR на датасете

In [75]:
res_correct = []
res_rr = []

# я понимаю, что использовать по одному на gpu не лучшая идея, но на cpu сойдет
for example in tqdm(new_ds, total=len(new_ds)):
    c = example['context']
    q = example['question']
    ans = qa_pipeline(question=q,  context=c)
    payload = {
        "inputs": {
            "source_sentence": ans['answer'],
            "sentences": example['answers']
        }
    }
    res_correct.append(is_correct(payload, example['labels']))
    res_rr.append(rr(payload, example['labels']))

print(f"Accuracy: {np.mean(res_correct)}")
print(f"MRR: {np.mean(res_rr)}")

  0%|          | 9/2889 [00:00<03:48, 12.59it/s]/home/langley/miniconda3/lib/python3.11/site-packages/transformers/pipelines/base.py:1101: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(
100%|██████████| 2889/2889 [03:09<00:00, 15.27it/s]

Accuracy: 0.7691242644513673
MRR: 0.8705434406368986


Сравним со случаем когда мы вибираем случайно

In [48]:
# тут можно сделать это все аналитически (без подстановки в модель), но я этого не сделал (

res_correct = []
res_rr = []

for example in tqdm(new_ds, total=len(new_ds)):
    payload = {
        "inputs": {
            "source_sentence": np.random.choice(example['answers']),
            "sentences": example['answers']
        }
    }
    res_correct.append(is_correct(payload, example['labels']))
    res_rr.append(rr(payload, example['labels']))

print(f"Accuracy: {np.mean(res_correct)}")
print(f"MRR: {np.mean(res_rr)}")

100%|██████████| 2889/2889 [00:33<00:00, 86.82it/s]

Accuracy: 0.4648667358947733
MRR: 0.6862524518287758


Результаты сравнения моделей для поиска ответа на вопрос в контексте:

| Model | Accuracy | MRR |
| --- | --- | --- |
| [timpal0l/mdeberta-v3-base-squad2](https://huggingface.co/timpal0l/mdeberta-v3-base-squad2) | 0.7785 | 0.8748 |
| [AlexKay/xlm-roberta-large-qa-multilingual-finedtuned-ru](https://huggingface.co/AlexKay/xlm-roberta-large-qa-multilingual-finedtuned-ru) | 0.7695 | 0.8696 |
| [squad_ru_bert](https://docs.deeppavlov.ai/en/0.9.0/features/models/squad.html) от deeppavlov | 0.7681 | 0.8687 |
| Случайно выбираем ответ | 0.4600 | 0.6885 |

Собственно оставим верхнюю модель, тк она показала лучший результат


# Обработка запросов

### Вопрос + Документ

Выше я разобрал случай когда подается запрос вида: Вопрос + Документ. Его можно обрабатывать вот так:

In [54]:
context = "Берлин — столица Германии, один из 16 штатов страны. Население города составляет 3,7 млн человек, что делает его самым населённым городом страны. Берлин является крупнейшим городом страны по площади (891,85 км²). Берлин расположен в восточной части страны, в 70 км от границы с Польшей. Город является одним из крупнейших транспортных узлов Европы, в нём находятся 2 аэропорта, 2 международных вокзала, 6 автовокзалов, а также крупнейший в Европе порт на реке Хафель."
question = "Сколько аэропортов в Берлине?"

ans = qa_pipeline(question=question, context=context)
ans['answer']

' 2'

### Only Вопрос

Другой случай, когда нам необходимо отвечать на вопрос без предоставленного заранее документа, а по какой-то коллекции документов. Для этого предложим такую схему:

- Индексируем документы
- Предобрабатываем запрос
- Ищем подходящие документы по максимальному cosine similarity
- Конкатенируем найденные документы в один большой документ
- Ищем ответ в этом большом документе

Необходимо найти модель которая будет хорошо работать на этапе поиска подходящих документов. Для этого я сделал следующий эксперимент:

- Взял все 500 документов из датасета
- Взял 500 случайных вопросов из датасета. Один вопрос - на один документ
- Предобработал документы и вопросы
- Посчитал косинусное сродство между вопросом и документом
- Посмотрел в насколько хорошо модель находит подходящий документ в топе из K документов (accuracy@K)

In [30]:
# сэмплирование документов
embedder = SentenceTransformer('cointegrated/rubert-tiny2')

corpus = [example['passage']['text'] for example in ds]
corpus_embeddings = embedder.encode(corpus, convert_to_tensor=True)

# сэмплирование запросов
queries = [example['passage']['questions'][0]['question'] for example in ds]
query_embeddings = embedder.encode(queries, convert_to_tensor=True)

# подсчет косинусного расстояния
cos_scores = util.cos_sim(query_embeddings, corpus_embeddings).cpu().numpy()
assert cos_scores.shape[0] == cos_scores.shape[1]

y_true = np.arange(len(queries))

# подсчет метрики
top_k_accuracy_score(y_true, cos_scores, k=5)

0.712

Результаты:

model | Accuracy@1 | Accuracy@5 | Accuracy@10 |
| --- | --- | --- | --- |
| [rubert-tiny2](https://huggingface.co/cointegrated/rubert-tiny2) | 0.494 | 0.712 | 0.77 | 
[paraphrase-multilingual-mpnet-base-v2](https://huggingface.co/sentence-transformers/paraphrase-multilingual-mpnet-base-v2) | 0.528 | 0.684 | 0.758 |

Будем юзать [rubert-tiny2](https://huggingface.co/cointegrated/rubert-tiny2) и возвращать 5 самых подходящих документов. Больше нет смысла тк максимально поддерживаемое количество токенов у этой модели 2048, что примерно 5-7 документов в зависимости от длины документа (средняя длина документа ~320 токенов).

Писать функции для индексирование не нужно, я использовал библиотеку [langchain](https://www.langchain.com/). С помощью [Annoy](https://python.langchain.com/docs/integrations/vectorstores/annoy) я индексировал документы и находил в полученной базе данных подходящие для ответа на вопрос документы.

In [31]:
model_name = "cointegrated/rubert-tiny2"
model_kwargs = {'device': 'cuda:0'}
encode_kwargs = {'normalize_embeddings': False}
embeddings_function = HuggingFaceEmbeddings(
    model_name=model_name,
    model_kwargs=model_kwargs,
    encode_kwargs=encode_kwargs
)


db = Annoy.from_texts(corpus, embeddings_function)

In [32]:
q = "Где бегала шпана?"

docs = db.similarity_search(q, k=5)
c = " ".join([d.page_content for d in docs])

ans = qa_pipeline(question=q,  context=c)
print('Контекст: ', c)
print('Вопрос: ', q)
print('Ответ: ', ans['answer'])

Контекст:  (1) Стояло жаркое лето, столько работы вокруг! (2) Белочка трудилась с утра и до вечера, не покладая рук. (3) Ей даже перекусить было некогда, только и делала, что таскала шишки в свое дупло. (4) Но места в дупле оставалось не так уж и много, ведь всё дупло было завалено припасами с прошлых лет. (5) Когда шишки совсем перестали помещаться в дупло, белочка загрустила: что же теперь делать? (6) Куда складывать питание? (7) Пошла белочка к дятлу и стала просить его расширить ей дупло. (8) - Ну, пожалуйста, дорогой дятлик, выдолби мне дупло побольше, а то маленькое оно у меня, ничего в нем не помещается! (9) Дятел был добрым, потому и согласился. (10) Залез он к белке в дупло, смотрит, а дупло-то у нее огромное! (11) И еды в нем видимо-невидимо. (12) Хватит, чтобы весь лес прокормить на несколько лет вперед! (13) Выдолбил дятел белочке дупло и говорит: - Что ж ты гонишь от себя зверушек голодных? (14) Еды-то у тебя на всех хватит! (15) Ты бы не жадничала, белочка, гляди, все теб

Saving database to disk

In [33]:
db.save_local('train_db')

Loading database from disk

In [34]:
db = Annoy.load_local('train_db', embeddings_function)

Реализация в streamlit приложении

# Выводы

- Наиболее подходящей моделью для поиска ответа на вопрос в контексте оказалась [timpal0l/mdeberta-v3-base-squad2](https://huggingface.co/timpal0l/mdeberta-v3-base-squad2). Она показала accuracy 0.7785 и MRR 0.8748. Это лучший результат среди всех моделей, которые я пробовал.

- Для поиска подходящих документов я использовал модель [rubert-tiny2](https://huggingface.co/cointegrated/rubert-tiny2). Она показала accuracy@5 0.712. То есть в 71% случаев подходящий документ находится в топе из 5 документов.

- Для правильно вытащенного документа с вероятностью 78% модель находит правильный ответ